# CS 675 Machine Learning
# Human Protein Atlas - Single Cell Classification
# Find individual human cell differences in microscope images
# Supervised Learning models (Decision Tree and KNN)
## Team Members are pr454(Preethi Ravulapally), sk2924(Kopparapu Sai Venkat Krishna) and pb536(Pooja Reddy Banala)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [261]:
# Type of Problem:  Multiple label Predictions per Single Image
# pip install torch
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,r2_score
import torch
from tqdm import tqdm
from sklearn.tree import DecisionTreeClassifier
import torch
import torchvision 
from torchvision.models import vgg16
from torchvision.transforms import Compose,Resize,RandomResizedCrop,Normalize,ToTensor
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset,DataLoader
import os
import torch.nn as nn
from PIL import Image
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torchvision.models as models
from sklearn.metrics import f1_score
from torchvision.utils import make_grid
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, random_split, DataLoader

%matplotlib inline


## Loading the train.csv file

In [262]:
dataset = pd.read_csv("/content/train.csv")
dataset.head()

,ID,Label
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0


In [263]:
labels = {
0:'Nucleoplasm',
1:'Nuclear membrane',
2:'Nucleoli',
3:'Nucleoli fibrillar center',
4:'Nuclear speckles',
5:'Nuclear bodies',
6:'Endoplasmic reticulum',
7:'Golgi apparatus',
8:'Intermediate filaments',
9:'Actin filaments',
10:'Microtubules',
11:'Mitotic spindle',
12:'Centrosome',
13:'Plasma membrane',
14:'Mitochondria',
15:'Aggresome',
16:'Cytosol',
17:'Vesicles and punctate cytosolic patterns',
18:'Negative'
}

## Preprocessing the csv file

Separating the labels for each image id. Based on the existing labels mark it as 1 and remaining as 0.

In [217]:
import pandas as pd
import numpy as np
from PIL import Image
from pathlib import Path

img_size = 256

classes = ["Nucleoplasm", "Nuclear Membrane", "Nucleoli", "Nucleoli Fibrillar Center", "Nuclear Speckles", "Nuclear Bodies", "Endoplasmic Reticulum", "Golgi Apparatus", "Intermediate Filaments", "Actin Filaments", "Microtubules", "Mitotic Spindle", "Centrosome", "Plasma Membrane", "Mitochondria", "Aggresome", "Cytosol", "Vesicles", "Negative"]
class2idx = {i :cls for i,cls in enumerate(classes)}
idx2class = {cls:i  for i,cls in enumerate(classes)}
    
path = "/content/train.csv"
df = pd.read_csv(path)

# df.apply(get_x, axis = 1) # Run if you dont have the resized Data

for i in range(len(classes)):
    df[f'{i}'] =0

for i in range(df.shape[0]):
    a = df.loc[i,'Label'].split('|')
    a_len = len(df.loc[i,'Label'].split('|'))
    for j in range(a_len ):
        df.loc[i,a[j]] = 1

In [218]:
df.drop(['Label'],axis=1).to_csv('/content/train_cleaned.csv',index=False)

In [219]:
# Dataset Stats: 
print("Dataframe Statistics:",)
df.drop(['Label','ID'],axis=1).describe()

Dataframe Statistics:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000,21806.000000
mean,0.403421,0.050216,0.112400,0.057874,0.065349,0.082179,0.035541,0.084656,0.044208,0.045767,0.037513,0.003577,0.079519,0.142667,0.092314,0.011556,0.260708,0.027194,0.001559
std,0.490595,0.218394,0.315866,0.233510,0.247146,0.274644,0.185146,0.278375,0.205562,0.208985,0.190019,0.059702,0.270554,0.349741,0.289475,0.106880,0.439031,0.162653,0.039457
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [220]:
# Total No of Items : 
# Total No of Samples in Each Class 
print("Total number of Samples:", df.shape[0])

for i in range(len(classes)):
    print(f"% of Samples in {classes[i]}: {df[f'{i}'].mean()}")

df.head()


Total number of Samples: 21806
% of Samples in Nucleoplasm: 0.40342107676786204
% of Samples in Nuclear Membrane: 0.05021553700816289
% of Samples in Nucleoli: 0.11240025681005228
% of Samples in Nucleoli Fibrillar Center: 0.05787397963863157
% of Samples in Nuclear Speckles: 0.06534898651747226
% of Samples in Nuclear Bodies: 0.08217921672934055
% of Samples in Endoplasmic Reticulum: 0.035540676877923505
% of Samples in Golgi Apparatus: 0.08465559937631845
% of Samples in Intermediate Filaments: 0.044208016142346145
% of Samples in Actin Filaments: 0.04576722003118408
% of Samples in Microtubules: 0.03751261120792442
% of Samples in Mitotic Spindle: 0.00357699715674585
% of Samples in Centrosome: 0.07951939833073467
% of Samples in Plasma Membrane: 0.142667155828671
% of Samples in Mitochondria: 0.09231404200678713
% of Samples in Aggresome: 0.011556452352563515
% of Samples in Cytosol: 0.26070806200128405
% of Samples in Vesicles: 0.027194350178849858
% of Samples in Negative: 0.0015

,ID,Label,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Drop this Label and save this to train_cleaned.csv 

In [133]:
import torch
import torchvision 
from torchvision.models import vgg16
from torchvision.transforms import Compose,Resize,RandomResizedCrop,Normalize,ToTensor

# tfms : 224x224x3 Size ? 
# Augmentation ? 

tfms = Compose(
    [RandomResizedCrop((224,224)),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


# Cross Validation Splits : 
from sklearn.model_selection import train_test_split

df= pd.read_csv('/content/train_cleaned.csv')
dev_df ,test_df = train_test_split(df,test_size=0.1,random_state=42)
train_df ,val_df = train_test_split(dev_df,test_size=0.1,random_state=42)



In [134]:
from PIL import Image
from torch.utils.data import Dataset,DataLoader
class CSVDataset(Dataset):
    def __init__(self, path,df,  tfms):
        self.path = path
        self.df = df
        self.transform = tfms
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, index):
        ID,*labels = self.df.iloc[index]   # Unpack labels into labels variable
        labels =np.array(labels)
        # I take just a "green" images
        file_path = Image.open(self.path + ID + '.png')
        if self.transform is not None:
            return self.transform(file_path), (ID,labels)
        else:
            return ValueError("Pass Transforms Correctly")
        
trainPath = "/content/ML/train/"
train_ds = CSVDataset(path=trainPath,df=train_df,tfms=tfms)
val_ds = CSVDataset(path=trainPath,df=val_df,tfms=tfms)
test_ds = CSVDataset(path=trainPath,df=test_df,tfms=tfms)


In [140]:
# Use VGG16 as Feature Extractor
device = 'cuda' if torch.cuda.is_available() else 'cpu'

vgg16_model = vgg16(pretrained=True)
vgg16_model.classifier = vgg16_model.classifier[:-1] 

vgg16_model= vgg16_model.eval().to(device)


train_dl = DataLoader(train_ds,batch_size=32,shuffle=False, pin_memory=True,num_workers=4)
val_dl = DataLoader(val_ds,batch_size=32,shuffle=False, pin_memory=True,num_workers=4)
test_dl = DataLoader(test_ds,batch_size=32,shuffle=False, pin_memory=True,num_workers=4)

# Test Random Inputs
inp = torch.randn((1,3,224,224))
feat_df = pd.DataFrame(columns=['ID','feature_vec'])

from tqdm import tqdm

file_ids =[]
feature_vecs =[]
label_vecs =[]

dls = {'train':train_dl , 'valid':val_dl }

for ds_split, dl in dls.items(): 
    with torch.no_grad():
        for img, meta_data in tqdm(dl):
            img = img.to(device)
            file_id , labels = meta_data
            feature_vec = vgg16_model(img).squeeze(0)
            file_ids.extend(file_id)
            feature_vecs.extend(feature_vec.cpu().numpy())
            label_vecs.extend(labels.cpu().numpy())
    torch.save({'ID':file_ids,'feature_vec':feature_vecs,'label_vec':label_vecs},f'{ds_split}.pt')


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



  0%|          | 0/552 [00:00<?, ?it/s]


  0%|          | 1/552 [00:01<09:10,  1.00it/s]


  0%|          | 2/552 [00:01<06:41,  1.37it/s]


  1%|          | 3/552 [00:01<04:57,  1.85it/s]


  1%|          | 5/552 [00:01<04:05,  2.23it/s]


  1%|▏         | 7/552 [00:01<03:06,  2.92it/s]


  2%|▏         | 9/552 [00:02<02:41,  3.36it/s]


  2%|▏         | 11/552 [00:02<02:08,  4.23it/s]


  2%|▏         | 13/552 [00:02<02:04,  4.34it/s]


  3%|▎         | 14/552 [00:02<01:44,  5.13it/s]


  3%|▎         | 15/552 [00:03<01:30,  5.95it/

In [141]:
torch.save({'ID':file_ids,'feature_vec':feature_vecs,'label_vec':label_vecs},'train.pt')


In [103]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,r2_score
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier

train_pt= torch.load('train.pt')
valid_pt = torch.load('valid.pt')

X = train_pt['feature_vec']
Y = train_pt['label_vec']

X_val = valid_pt['feature_vec']
# Y_val = np.array(valid_pt['label_vec']).reshape(-1,len(classes))
X_train, X_Test, Y_train,Y_Test = train_test_split(X,Y,test_size=0.25);

## Decision Tree Classifier

In [104]:
clf = DecisionTreeClassifier(random_state=0)
multi_target_forest = MultiOutputClassifier(clf, n_jobs=-1)
multi_target_forest.fit(X_train, Y_train)

MultiOutputClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort='deprecated',
                                                       random_state=0,
                

I gave one test sample to predict and compare it with Y_test data. 

In [114]:
multi_target_forest.predict(X_Test[1].reshape(1,-1))

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [116]:
print(Y_Test[1])

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


Here we can see that this model is predicting multi labels. Y_Test[1] has the label as <b>12.Centrosome</b>. Map this output array with the labels.

## KNN Model

In [118]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
multi_label = MultiOutputClassifier(knn)
multi_label.fit(X_train, Y_train)

MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='minkowski',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=3,
                                                     p=2, weights='uniform'),
                      n_jobs=None)

In [123]:
multi_label.predict(X_Test[1].reshape(1,-1))

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [127]:
print(Y_Test[1])

[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


Here Two labels are predicted. The Labels are <b>0.Nucleoplasm</b> and <b>12.Centrosome</b>